In [1]:
from matplotlib import pyplot as plt
import platform, os, sys, pprint, h5py, emoji
from tqdm import tqdm
import numpy as np
from glob import glob
from tabulate import tabulate
from pprint import pprint
from importlib import reload
from mpl_toolkits.axes_grid.inset_locator import inset_axes

if 'Windows' in platform.system():
    sys.path.append(r'C:\Users\slab\Documents\Code')
    sys.path.append(r'D:\BEMPP_shared\Modules')
    import interpolate_slow
else:
    sys.path.append("/Users/gkoolstra/Documents/Code")
    from BEMHelper import interpolate_slow
    
from Common import common, kfit
from TrapAnalysis import trap_analysis
from TrapAnalysis import artificial_anneal as anneal

%matplotlib inline
%config InlineBackend.figure_format='retina'

Could not import BEM module. Please do so manually.


# 2 Subplots with the chemical potential

In [3]:
import matplotlib.gridspec as gridspec
%matplotlib inline
#%matplotlib notebook

save_path = r"/Users/gkoolstra/Desktop/Single electron loading/Data"
sub_dir = r"170314_222917_M018V6_Greater_Trap_Area_V6.2_resonator_sweep"

dbin = 0.006E-6
bins = np.arange(-0.80E-6, 0.80E-6+dbin, dbin)
save = False
xbounds = (-3, 9)

converged, energy, num_trapped_electrons = list(), list(), list()
with h5py.File(os.path.join(os.path.join(save_path, sub_dir), "Results.h5"), "r") as f:
    print("Directory: %s" % sub_dir)
    k = 0
    
    Vresguard = f["Vrg"][()]
    trap_grad_x_i, trap_grad_x_f, trap_grad_y_i, trap_grad_y_f = list(), list(), list(), list()
    
    resVs, trapVs, resgVs, trapgVs = f["Vres"][()], f["Vtrap"][()], f["Vrg"][()], f["Vtg"][()]
    trap_sweep_idx = np.where(np.diff(trapVs, n=2) != 0)[0]
    trapg_sweep_idx = np.where(np.diff(trapgVs, n=2) != 0)[0]
    resg_sweep_idx = np.where(np.diff(resgVs, n=2) != 0)[0]
    
    #print(trap_sweep_idx)
    #print(trapg_sweep_idx)
    #print(resg_sweep_idx)
    
    for step in f.keys():
        if "step" in step:
            electron_ri = f[step + "/electron_final_coordinates"][()]
            xi, yi = anneal.r2xy(electron_ri)
            energy.append(f[step+"/final_energy"][()])

            valid_solution = f[step + "/solution_converged"][()]
            converged.append(valid_solution)

            if valid_solution:
                electron_hist, bin_edges = np.histogram(xi, bins=bins)
            else:
                electron_hist = np.zeros(len(electron_hist))

            if k == 0:
                electron_histogram = electron_hist
            else:
                electron_histogram = np.vstack((electron_histogram, electron_hist))

            k += 1
    
    if 0:
        step = 201
        rs = f['step_%04d/x_during_simulation'%step]
        jac = f['step_%04d/jac_during_simulation'%step]

        plt.figure()
        plt.plot(np.amax(jac[:,::2], axis=1), label='grad_x')
        plt.plot(np.amax(jac[:,1::2], axis=1), label='grad_y')
        plt.legend(loc=0)
        plt.yscale('log')
        
        for k in tqdm(range(np.shape(rs)[0])):
            fig = plt.figure(figsize=(6.5,3.))
            plt.plot(rs[k,::2]*1E6, rs[k,1::2]*1E6, 'o', color='deepskyblue')
            plt.xlim(-4, 8)
            plt.ylim(-4, 4)
            plt.ylabel("$y$ ($\mu$m)")
            plt.title("Resonator guard = %.3f V"%Vresguard[step])
            common.save_figure(fig, save_path=os.path.join(save_path, sub_dir, "Annealing"))
            plt.close(fig)

    if 1:
        f0 = list()
        trap = list()
        xpoints, ypoints = f["xpoints"][()], f["ypoints"][()]
        for step in tqdm(range(259,373)):#tqdm(range(k)):
            # This assumes that we sweep the resonator guard.
            Vres, Vtrap, Vrg, Vtg = f["Vres"][()][step], f["Vtrap"][()][step], f["Vrg"][()][step], f["Vtg"][()][step]
            
            potential = Vres * f["resonator"][()] + Vtrap * f["trap"][()] + Vrg * f["res_guard"][()] + \
                        Vtg * f["trap_guard"][()]
            
            electrons = f["step_%04d"%step + "/electron_final_coordinates"][()]

            fig = plt.figure(figsize=(6.5, 7.5))
            plt.subplot(211)
            common.configure_axes(13)
            colors = plt.pcolormesh(xpoints*1E6, ypoints*1E6, potential, 
                                    cmap=plt.cm.RdYlBu, vmin=-0.70, vmax=+0.30)
            
            TS = anneal.TrapAreaSolver(xpoints[0,:], ypoints[:,0], potential.T, smoothing=0.01)
            T = trap_analysis.TrapSolver()

            PP = anneal.PostProcess()
            PP.draw_from_dxf(filename=r"/Users/gkoolstra/Desktop/Single electron loading/Potentials/M018V6/V6.2/all_electrodes.dxf", 
                             **{'color' : 'black', 'lw' : 0.75, 'alpha' : 0.75})

            plt.plot(electrons[::2]*1E6, electrons[1::2]*1E6, 'o', color='deepskyblue')
            plt.xlim(xbounds[0], xbounds[1])
            plt.ylim(-4, 4)
            plt.ylabel("$y$ ($\mu$m)")
            
            fd = {'rotation':90, 'ha' : 'center', 'va' : 'center', 'color' : 'white', 'size' : 15}
            if step >= 0 and step < trap_sweep_idx[0]:
                plt.text(1.5, 2.5, "- - -", fontdict=fd)
                plt.text(1.5, -2.5, "- - -", fontdict=fd)
            
            if step >= trapg_sweep_idx[0] and step < trap_sweep_idx[0] : 
                plt.text(-1.5, 2.5, "+ + +", fontdict=fd)
                plt.text(-1.5, -2.5, "+ + +", fontdict=fd)
            if step >= trap_sweep_idx[0]: 
                plt.text(-2.5, 0, "+ + +", fontdict=fd)
                
                # There is a single electron in the trap, select it's x coordinate
                #mask = electrons[::2] < 1E-6
                #x_electron = electrons[::2][mask]
                
                fr, ferr = T.fit_electron_potential(ypoints[:,0] * 1E6, -TS.V(0, ypoints[:,0]), 
                                                    fitdomain=(-0.25, 0.25), do_plot=False, plot_title='')
                f_0, sigmaf = T.get_electron_frequency(fr, ferr, verbose=False)
                
                trap.append(Vtrap)
                f0.append(f_0)
            
            #plt.title("Resonator guard = %.3f V"%Vrg)

            ctr_idx = int(np.shape(ypoints)[0]/2)
            idcs = list()
            for xs in electrons[::2]: 
                idcs.append(common.find_nearest(xpoints[ctr_idx], xs))

            plt.subplot(212)
            

            electrons_x, electrons_y = electrons[::2], electrons[1::2]
            #TS = anneal.TrapAreaSolver(xpoints[0,:], ypoints[:,0], potential.T)
            interactions = TS.Vee(electrons_x, electrons_y)/1.602E-19
            np.fill_diagonal(interactions, 0)
            mu = list()
            el = 0
            for electron_x, electron_y in zip(electrons[::2], electrons[1::2]):
                mu.append(np.sum(interactions[el,:]) + TS.Velectrostatic(electron_x, electron_y)/1.602E-19)
                el += 1
            
            plt.plot(xpoints[ctr_idx,:]*1E6, TS.V(xpoints[ctr_idx,:], 0), color='k')
            #plt.plot(xpoints[ctr_idx,:]*1E6, potential[ctr_idx,:], color='k')
            plt.plot(electrons_x*1E6, np.array(mu), 'o', color='deepskyblue')
            
            plt.xlim(xbounds[0], xbounds[1])
            plt.xlabel("$x$ ($\mu$m)")
            plt.ylabel("Electrostatic potential (eV)")
            #plt.title("Resonator guard = %.3f V"%Vrg)
            plt.ylim(-0.7, -0.44)
            
            if step >= trap_sweep_idx[0]: 
                # Trap deformation and cavity frequency: 
                #get_eigenfreqencies(self, alpha, beta, f0)
                
                # Inset of electron frequency:
                insax = inset_axes(plt.gca(), width="37%", height=1.5, loc=1)
                #a = plt.axes([.55, .65, .3, .2])
                #common.configure_axes(11)
                insax.plot(np.array(trap), np.array(f0)/1E9, '-k', lw=3.0)
                plt.xlabel("Trap voltage (V)")
                plt.ylabel("Electron freq. (GHz)")
                plt.xlim(trapVs[0], trapVs[-1])
                plt.xticks([0.95, 0.85, 0.75])
                plt.ylim(0, 30)
                plt.yticks([0, 10, 20, 30])
            
            common.save_figure(fig, save_path=os.path.join(save_path, sub_dir, "2D slice"))
            plt.close('all')

    
print("\tOut of %d simulations, %d did not converge..." % (len(Vresguard), len(Vresguard) - np.sum(converged)))

Directory: 170314_222917_M018V6_Greater_Trap_Area_V6.2_resonator_sweep


 87%|████████▋ | 99/114 [09:31<01:26,  5.79s/it]/Users/gkoolstra/anaconda/lib/python3.5/site-packages/scipy/optimize/minpack.py:715: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
 97%|█████████▋| 111/114 [10:41<00:17,  5.81s/it]/Users/gkoolstra/Documents/Code/TrapAnalysis/trap_analysis.py:169: RuntimeWarning: invalid value encountered in sqrt
  f = 1/(2*np.pi) * np.sqrt(-c['e']*2*fr[1]*1E12/c['m_e'])
/Users/gkoolstra/Documents/Code/TrapAnalysis/trap_analysis.py:170: RuntimeWarning: invalid value encountered in sqrt
  sigma_f = 1/(4*np.pi) * np.sqrt(-2*c['e']*1E12/(fr[1]*c['m_e'])) * ferr[1]
100%|██████████| 114/114 [10:59<00:00,  5.92s/it]

	Out of 373 simulations, 0 did not converge...


# Effect of smoothing on the potential

In [ ]:
smoothing = 1E-2
TS = anneal.TrapAreaSolver(xpoints[0,:], ypoints[:,0], potential.T, smoothing=smoothing)
plt.pcolormesh(xpoints[0,:] * 1E6, ypoints[:,0] * 1E6, 
               np.log10(np.abs(potential - np.reshape(TS.V(xpoints.flatten(), ypoints.flatten()), np.shape(xpoints)))), 
               cmap=plt.cm.RdYlBu)
plt.colorbar()
plt.title(r"$^{10}\log |\Delta| $ for $s=$%.1e"%(smoothing))
plt.clim(-6, -3)
plt.xlim(-2, 6)
plt.ylim(-4, 4)

In [ ]:
%matplotlib inline
plt.figure()
plt.plot(xpoints[0,:]*1E6, potential [int(np.shape(ypoints)[0]/2),:])
plt.plot(xpoints[0,:]*1E6, TS.V(xpoints[0,:],0), 'r')
plt.xlim(-1, 1)
plt.ylim(-0.619, -0.618)

In [ ]:
%matplotlib inline
plt.figure()
plt.plot(xpoints[0,:]*1E6, TS.dVdx(xpoints[0,:],0), 'r')
plt.xlim(-1, 1)
plt.ylim(-1e5, 1e5)

In [ ]:
plt.figure()
plt.text(0.5, 0.5, "+ + +", fontdict={'rotation':90, 'ha' : 'center', 'va' : 'center'})

In [6]:
import json, pprint

In [18]:
with open(r"/Users/gkoolstra/Documents/Code/iPythonNotebooks/Scripts/settings.json") as data_file:    
    data = json.load(data_file)

pprint.pprint(data)

JSONDecodeError: Expecting value: line 10 column 35 (char 290)

In [11]:
data['maps']

[{'id': 'blabla', 'iscategorical': '0'},
 {'id': 'blabla', 'iscategorical': '0'}]